In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tools import Tools
from model import Model
import json
from test import test
import matplotlib.pylab as plt

In [4]:
def get_data(index, cf):
    begin_cf = cf.columns.get_loc("0")
    data_array = cf.iloc[index, begin_cf:begin_cf+167].values
    N = len(data_array)

    nr_joints = len(eval(data_array[0]))

    data = np.zeros((N,nr_joints,2))

    for j in range(N):
        if not pd.isnull(data_array[j]):
            data[j]=np.array(eval(data_array[j]))
        else:
            data[j] = data[j-1]
    return data

In [ ]:
def get_joint_array(csv_file_path, files, player, dic_test):
    cf = pd.read_csv(csv_file_path)
    # cf = pd.read_csv("/Users/ninawiedemann/Desktop/UNI/Praktikum/ALL/cf_data.csv")
    cf_player = cf[cf["Player"]==player]
    games = cf_player["Game"].values.tolist()
    positions = cf_player["Pitching Position (P)"].values
    joints_array = []
    lab_test = []
    position = []
    # pitch_type = []
    for i, game in enumerate(files):
        # print(game)
        try:
            ind = games.index(game)
            position.append(positions[ind])
            #pitch_type.append(pitchtypes[ind])
            joints_array.append(get_data(ind, cf_player))
            lab_test.append(dic_test[game])
        except:
            continue
    return np.array(joints_array), lab_test

def testing(files, dic, restore_path):
    joints, labels = get_joint_array("/Users/ninawiedemann/Desktop/UNI/Praktikum/ALL/sv_data.csv", files, "Pitcher", dic)
    print(joints.shape)
    joints_rel = joints[:, :, [7,8,10,11],:]
    joints_rel = Tools.normalize(joints_rel)
    print(joints_rel.shape, np.array(labels).shape)
    data = []
    for bsp in joints_rel:
        for i in range(14):
            data.append(bsp[i*10:(i+2)*10])
    data = np.array(data)
    #np.save("/Users/ninawiedemann/Desktop/UNI/Praktikum/ALL/notebooks/bsp_data.npy", data)
    #np.save("/Users/ninawiedemann/Desktop/UNI/Praktikum/ALL/notebooks/bsp_labels.npy", labels)
    print(data.shape)
    # data = Tools.normalize(data)
    lab, out = test(data, restore_path)
    print("out", out.shape)
    # print(out)
    truth_val = out[:, 1]
    r = 0
    for l in range(len(labels)):
        res = []
        for i in range(14):
            res.append(truth_val[(l*14)+i])
        result = (np.argmax(res)+1)*10
        print("result ", result, labels[l], "label")
        if abs(result-labels[l])<10:
            r+=1
        else: 
            for jo in range(4):
                plt.plot(joints_rel[l,:,jo,0])
            plt.show()
            plt.title("Y")
            for jo in range(4):
                plt.plot(joints_rel[l,:,jo,1])
            plt.show()
            plt.title("X")
    print("Accuracy: ", r/float(len(labels)))

with open("/Users/ninawiedemann/Desktop/UNI/Praktikum/ALL/notebooks/first_move_frames.json", "r") as infile:
    dic_lab = json.load(infile)
all_files = list(dic_lab.keys())
print(len(all_files))
# files = all_files[:-10]
# print(len(all_files))
# print(all_files[:10])

testing(all_files, dic_lab, "/Users/ninawiedemann/Desktop/UNI/Praktikum/saved_models/first_move")

In [19]:
df = pd.read_csv("/Users/ninawiedemann/Desktop/UNI/Praktikum/ALL/sv_data.csv")
df = df[df["Player"]=="Batter"]
games = df["Game"].values
data_j = []
for i in range(len(df.values)):
    data_j.append(get_data(i, df))
data_j = np.array(data_j)
print(data_j.shape, games.shape)

/Users/ninawiedemann/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,

(4541, 167, 18, 2) (4541,)


In [31]:
ind_right = list(games).index("490509-3f640b3f-9ba5-4e89-8f72-97c9f333aa2c")
print(ind_right)
from scipy import ndimage
print(games[810])

809
490509-475e2c86-a1ba-42ff-8a3c-8737443a7cfa


In [48]:
smooth = ndimage.filters.gaussian_filter1d(data_j, axis =1, sigma = 3)
grads = np.gradient(smooth[:,:,:, 1], axis = 1)
mean_grads = np.mean(smooth[:,:,:,1], axis = 2)

for i in range(10):
    #print(i)
    lower = mean_grads[i][mean_grads[i]<0]
    cut = np.sum(lower[50:])
    if cut>0:
        print(games[i])
        for j in range(12):
            plt.plot(smooth[i,:,j,1])
        #plt.ylim(-4,4)
        plt.show()
        plt.plot(mean_grads[i])
        plt.ylim(-4,4)
        plt.show()